## PAIR PROGRAMMING ETL II

### Transformación I - Limpieza
---

In [43]:
import requests
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda


import ast

from datetime import datetime, timedelta

pd.options.display.max_columns = None

Tendréis que usar el csv attacks_limpieza_completa.

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.
2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El **objetivo** de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos

In [11]:
df= pd.read_csv('../files/attacks_limpieza_completa.csv', index_col=0)
df_clima = pd.read_csv('../files/clima_paises.csv', index_col=0)

- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea

In [12]:
df_attacks= df[df['country'].isin(['usa','australia','new zealand','south africa', 'papua new guinea'])]

In [13]:
df_attacks['country'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [14]:
df_attacks.shape

(4335, 21)

- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [15]:
df_clima.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,country,highcloud,midcloud,lowcloud,rh_profile,wind_profile,msl_pressure,snow_depth
0,3,1,15,none,0,12,4,NaN,270,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",1026.0,0.0
1,6,1,15,none,0,12,3,NaN,15,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",1027.0,0.0


- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
    - Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [16]:
df_clima.dtypes

timepoint              int64
cloudcover             int64
lifted_index           int64
prec_type             object
prec_amount            int64
temp2m                 int64
rh2m                   int64
weather              float64
wind10m.direction      int64
wind10m.speed          int64
country               object
highcloud            float64
midcloud             float64
lowcloud             float64
rh_profile            object
wind_profile          object
msl_pressure         float64
snow_depth           float64
dtype: object

In [24]:
df_clima['rh_profile']= df_clima['rh_profile'].apply(ast.literal_eval)

In [25]:
x = df_clima['rh_profile'].apply(pd.Series)

In [26]:
x.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 3}","{'layer': '900mb', 'rh': 0}","{'layer': '850mb', 'rh': -2}","{'layer': '800mb', 'rh': -1}","{'layer': '750mb', 'rh': 0}","{'layer': '700mb', 'rh': 2}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 7}","{'layer': '500mb', 'rh': 7}","{'layer': '450mb', 'rh': 5}","{'layer': '400mb', 'rh': 1}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': 2}","{'layer': '250mb', 'rh': 2}","{'layer': '200mb', 'rh': 4}"
1,"{'layer': '950mb', 'rh': 2}","{'layer': '900mb', 'rh': 0}","{'layer': '850mb', 'rh': 0}","{'layer': '800mb', 'rh': 1}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 5}","{'layer': '650mb', 'rh': 7}","{'layer': '600mb', 'rh': 7}","{'layer': '550mb', 'rh': 5}","{'layer': '500mb', 'rh': 3}","{'layer': '450mb', 'rh': 2}","{'layer': '400mb', 'rh': 0}","{'layer': '350mb', 'rh': 4}","{'layer': '300mb', 'rh': 5}","{'layer': '250mb', 'rh': 7}","{'layer': '200mb', 'rh': 7}"


In [31]:
for i in range(len(x.columns)): 
    

    # aplicamos el apply,extraemos el valor de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, valores)

In [ ]:
df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

In [35]:
y = df_clima['wind_profile'].apply(pd.Series)

In [ ]:
for i in range(len(y.columns)): 
    

    # aplicamos el apply,extraemos el valor de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "direction" + str(y[i].apply(pd.Series)["layer"][0]) 
    nombre2 = "speed" + str(y[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(y[i].apply(pd.Series)["direction"] )
    valores2= list(y[i].apply(pd.Series)["speed"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, valores)
    df_clima.insert(i,nombre2,valores2)

In [45]:
df_clima.head(3)

,speed950mb,speed900mb,speed850mb,speed800mb,speed750mb,speed700mb,speed650mb,speed600mb,speed550mb,speed500mb,speed450mb,speed400mb,speed350mb,speed300mb,speed250mb,speed200mb,direction200mb,direction250mb,direction300mb,direction350mb,direction400mb,direction450mb,direction500mb,direction550mb,direction600mb,direction650mb,direction700mb,direction750mb,direction800mb,direction850mb,direction900mb,direction950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,country,highcloud,midcloud,lowcloud,rh_profile,wind_profile,msl_pressure,snow_depth
0,2,2,3,4,4,5,5,5,5,6,6,6,7,7,8,8,340,335,335,330,325,310,305,305,310,315,320,315,305,300,250,210,3,0,-2,-1,0,2,4,6,7,7,5,1,-1,2,2,4,3,1,15,none,0,12,4,NaN,270,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",1026.0,0.0
1,2,2,3,3,3,4,5,5,5,5,6,7,7,7,7,8,340,335,325,325,330,325,310,305,305,310,310,315,325,355,35,345,2,0,0,1,3,5,7,7,5,3,2,0,4,5,7,7,6,1,15,none,0,12,3,NaN,15,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",1027.0,0.0
2,2,2,3,4,4,5,5,5,5,6,6,6,6,6,6,7,345,330,335,320,320,325,325,320,330,335,330,330,335,330,0,180,2,1,4,6,6,6,7,7,6,4,1,-1,1,3,6,5,9,1,15,none,0,12,2,NaN,310,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",1028.0,0.0


In [48]:
df_clima.drop(['rh_profile','wind_profile'], axis=1, inplace=True)# Eliminamos las columnas que tienen las listas de diccionarios.

In [49]:
df_clima.shape

(320, 64)

In [ ]:
df_group = df_clima.groupby

In [ ]:
# os recomendamos resetear el index del dataframe de los datos climáticos para que no se repitan los nombres de las columnas.


# El primer problema al que nos podemos enfrentar es que si vemos los tipos de las columnas vemos que estas columnas son objetos, es decir, strings, lo que hará que trabajar con ellas sea un poco complicado: 
clima.dtypes

timepoint             int64
cloudcover            int64
highcloud             int64
midcloud              int64
lowcloud              int64
rh_profile           object
wind_profile         object

# en Python tenemos la librería `ast` que nos permite castear un string que dentro tiene diccionarios, o listas o tuplas a su tipo correspondiente. En nuestro caso, lo que conseguiremos es no tener strings sino listas en la columna. Esto lo haremos de la siguiente forma: 

import ast

clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)

# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 


x = clima['wind_profile'].apply(pd.Series)


# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 
x = df['rh_profile'].apply(pd.Series) 

# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario (porque nuestra lista esta compuesta por distintos diccionarios)

# Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. Ahora tenemos que despempaquetar la información de los diccionarios en distintas columnas. En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los values los valores de las celdas del dataframe. Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 

# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 

- Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.